# Code for running inference

10-18-2024

To convert this to a script, should have a name for the whole training task which goes in dataset.json and/or datalist.json because this will help for the task.json needed for inference.

Look into the minimal amount of metaata monai requires in the form of a task.json or other struct to initialize the builder and then run the ensemble


In [8]:
import os
from pathlib import Path
import numpy as np
import json
from loguru import logger
import platform

import mri_data
import monai_training

from monai.apps.auto3dseg import (
    AlgoEnsembleBestN,
    AlgoEnsembleBuilder,
    import_bundle_algo_history,
)
from monai.utils.enums import AlgoKeys

from reload_recursive import reload_recursive

In [9]:
reload_recursive(mri_data)
reload_recursive(monai_training)
from mri_data.file_manager import scan_3Tpioneer_bids, DataSet, filter_first_ses  # noqa: E402, F401
from monai_training import preprocess  # noqa: E402


Setup logging


In [10]:
log_dir = ".logs"

if not os.path.exists(log_dir):
    os.makedirs(log_dir)
logger.add(
    os.path.join(log_dir, "file_{time:%Y_%m_%d}.log"), rotation="6h", level="DEBUG"
)

2

**Edit These**


In [11]:
#! Set these variables
work_dir_name = "choroid_pineal_pituitary3"
train_dataset_file_name = "training-dataset.json"
prediction_postfix = "choroid_pineal_pituitary3_pred"
task_name = "infer_choroid_pineal_pituitary"
modalities = ["flair", "t1"]

Set paths


In [12]:
hostname = platform.node()
if hostname == "rhinocampus" or hostname == "ryzen9":
    drive_root = Path("/media/smbshare")
else:
    drive_root = Path("/mnt/h")

projects_root = Path("/home/srs-9/Projects")

msmri_home = projects_root / "ms_mri"
training_work_dirs = msmri_home / "training_work_dirs"

# dataroot = "/media/hemondlab/Data/3Tpioneer_bids"
dataroot = drive_root / "3Tpioneer_bids"
work_dir = training_work_dirs / work_dir_name
train_dataset_file = work_dir / train_dataset_file_name
save_dir = drive_root / "3Tpioneer_bids_predictions"

prediction_filename = (
    ".".join(sorted(modalities)) + "_" + prediction_postfix + ".nii.gz"
)
print(prediction_filename)

taskfile_name = "inference-task.json"

flair.t1_choroid_pineal_pituitary3_pred.nii.gz


In [14]:
if not train_dataset_file.is_file():
    dataset = preprocess.parse_datalist(work_dir / "training-datalist.json", dataroot)
    preprocess.save_dataset(dataset, train_dataset_file)

2024-12-31 17:34:38.992 | INFO     | monai_training.preprocess:parse_datalist:282 - Loading /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary3/training-datalist.json
2024-12-31 17:34:38.993 | INFO     | monai_training.preprocess:parse_datalist:283 - /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary3/training-datalist.json exists: True


In [15]:
def inference_exists(dataset: DataSet) -> DataSet:
    count = 0
    dataset_new = DataSet(dataset.dataroot)
    for scan in dataset:
        if not (save_dir / scan.relative_path / prediction_filename).is_file():
            dataset_new.append(scan)
        else:
            count += 1
    logger.info(f"{count} scans already have inference")
    return dataset_new

### Do Preparation

Get all the scans that were not in the training and testing set to create the inference dataset


In [16]:
# the scans that were used in the training
dataset_train, _ = preprocess.load_dataset(train_dataset_file)
dataset_train.dataroot = dataroot

# dataset_train2 has the same subject/sessions that are in dataset_train but with a subset of the keys
#   so that they can be compared to scans in the full data set when getting the set difference
dataset_proc = preprocess.DataSetProcesser.new_dataset(
    dataroot, scan_3Tpioneer_bids, filters=[filter_first_ses, inference_exists]
)
dataset_full = dataset_proc.dataset
dataset_train2 = DataSet.dataset_like(dataset_train, ["subid", "sesid"])
dataset_inference = DataSet.from_scans(set(dataset_full) - set(dataset_train2))

2024-12-31 17:34:46.892 | INFO     | __main__:inference_exists:9 - 383 scans already have inference


Prepare the images in the inference dataset (i.e combine flair and t1 images)


In [17]:
dataset_proc = preprocess.DataSetProcesser(dataset_inference)
dataset_proc.prepare_images(["flair", "t1"])
dataset_proc.dataset.sort(key=lambda s: s.subid)

2024-12-31 17:34:52.575 | INFO     | monai_training.preprocess:prepare_images:107 - Prepare Images
  0%|          | 0/152 [00:00<?, ?it/s]2024-12-31 17:34:52.638 | INFO     | mri_data.utils:merge_images:31 - fslmerge -a /media/smbshare/3Tpioneer_bids/sub-ms2120/ses-20170920/flair.t1.nii.gz /media/smbshare/3Tpioneer_bids/sub-ms2120/ses-20170920/flair.nii.gz /media/smbshare/3Tpioneer_bids/sub-ms2120/ses-20170920/t1.nii.gz


Image Exception : #22 :: Failed to read volume /media/smbshare/3Tpioneer_bids/sub-ms2120/ses-20170920/flair.nii.gz
Error : Error: short read, file may be truncated
terminate called after throwing an instance of 'std::runtime_error'
  what():  Failed to read volume /media/smbshare/3Tpioneer_bids/sub-ms2120/ses-20170920/flair.nii.gz
Error : Error: short read, file may be truncated


2024-12-31 17:34:54.074 | ERROR    | monai_training.preprocess:prepare_images:147 - Something went wrong merging images for <bound method Scan.info of Scan(subid='2120', sesid='20170920', _dataroot=PosixPath('/media/smbshare/3Tpioneer_bids'), _root=PosixPath('/media/smbshare/3Tpioneer_bids/sub-ms2120/ses-20170920'), image=None, label=None, cond=None, id=42762350400)>
 17%|█▋        | 26/152 [00:01<00:07, 17.36it/s]2024-12-31 17:34:54.088 | WARNING  | monai_training.preprocess:prepare_images:138 - Couldn't prepare image for Scan(subid=1394, sesid=20191012)
2024-12-31 17:34:54.126 | WARNING  | monai_training.preprocess:prepare_images:138 - Couldn't prepare image for Scan(subid=2001, sesid=20161222)
 45%|████▌     | 69/152 [00:01<00:01, 53.24it/s]2024-12-31 17:34:54.191 | WARNING  | monai_training.preprocess:prepare_images:138 - Couldn't prepare image for Scan(subid=1364, sesid=20170705)
2024-12-31 17:34:54.196 | WARNING  | monai_training.preprocess:prepare_images:138 - Couldn't prepare i

/home/srs-9/fsl/share/fsl/bin/fslmerge: line 2: 2484762 Aborted                 (core dumped) /home/srs-9/fsl/bin/fslmerge "$@"
Image Exception : #22 :: Failed to read volume /media/smbshare/3Tpioneer_bids/sub-ms1196/ses-20161004/t1.nii.gz
Error : Error: short read, file may be truncated
terminate called after throwing an instance of 'std::runtime_error'
  what():  Failed to read volume /media/smbshare/3Tpioneer_bids/sub-ms1196/ses-20161004/t1.nii.gz
Error : Error: short read, file may be truncated


2024-12-31 17:34:57.221 | ERROR    | monai_training.preprocess:prepare_images:147 - Something went wrong merging images for <bound method Scan.info of Scan(subid='1196', sesid='20161004', _dataroot=PosixPath('/media/smbshare/3Tpioneer_bids'), _root=PosixPath('/media/smbshare/3Tpioneer_bids/sub-ms1196/ses-20161004'), image=None, label=None, cond=None, id=24112560784)>
2024-12-31 17:34:57.230 | WARNING  | monai_training.preprocess:prepare_images:138 - Couldn't prepare image for Scan(subid=2106, sesid=20170205)
2024-12-31 17:34:57.247 | WARNING  | monai_training.preprocess:prepare_images:138 - Couldn't prepare image for Scan(subid=1379, sesid=20161022)
 62%|██████▎   | 95/152 [00:04<00:03, 17.76it/s]2024-12-31 17:34:57.297 | WARNING  | monai_training.preprocess:prepare_images:138 - Couldn't prepare image for Scan(subid=2085, sesid=20170130)
2024-12-31 17:34:57.340 | WARNING  | monai_training.preprocess:prepare_images:138 - Couldn't prepare image for Scan(subid=1245, sesid=20220219)
100%|█

/home/srs-9/fsl/share/fsl/bin/fslmerge: line 2: 2484798 Aborted                 (core dumped) /home/srs-9/fsl/bin/fslmerge "$@"


Save the datalist and task files


In [18]:
images = []
for scan in dataset_proc.dataset:
    infile: Path = scan.image_path
    images.append({"image": str(infile.relative_to(dataset_proc.dataset.dataroot))})

logger.info(f"Will run inference on {len(images)} scans")

datalist = {"testing": images}

datalist_file = work_dir / "datalist.json"
with open(datalist_file, "w") as f:
    json.dump(datalist, f, indent=4)

task = {
    "name": task_name,
    "task": "segmentation",
    "modality": "MRI",
    "datalist": str(work_dir / "datalist.json"),
    "dataroot": str(dataroot),
}

task_file = os.path.join(work_dir, taskfile_name)
with open(task_file, "w") as f:
    json.dump(task, f, indent=4)

2024-12-31 17:35:07.357 | INFO     | __main__:<module>:6 - Will run inference on 142 scans


In [19]:
input_cfg = task_file  # path to the task input YAML file created by the users

history = import_bundle_algo_history(work_dir, only_trained=True)

In [21]:
## model ensemble
n_best = 5
builder = AlgoEnsembleBuilder(history, input_cfg)
builder.set_ensemble_method(AlgoEnsembleBestN(n_best=n_best))
ensemble = builder.get_ensemble()
save_params = {
    "_target_": "SaveImage",
    "output_dir": save_dir,
    "data_root_dir": dataroot,
    "output_postfix": prediction_postfix,
    "separate_folder": False,
}

pred = ensemble(pred_param={"image_save_func": save_params})

Ensembling (rank 0)...:   0%|          | 0/141 [00:00<?, ?it/s]You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
`torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast(

2024-12-31 17:36:31,900 INFO image_writer.py:197 - writing: /media/smbshare/3Tpioneer_bids_predictions/sub-ms2036/ses-20170520/flair.t1_choroid_pineal_pituitary3_pred.nii.gz


Image save path not returned.
Ensembling (rank 0)...:   1%|          | 1/141 [00:48<1:54:09, 48.93s/it]You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
`torch.cuda.amp.autocast(args...)` is dep

2024-12-31 17:37:32,791 INFO image_writer.py:197 - writing: /media/smbshare/3Tpioneer_bids_predictions/sub-ms2039/ses-20170118/flair.t1_choroid_pineal_pituitary3_pred.nii.gz


Ensembling (rank 0)...:   1%|▏         | 2/141 [01:49<2:09:40, 55.97s/it]You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
`torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp

...

Ensembling (rank 0)...:  99%|█████████▊| 139/141 [2:26:46<02:04, 62.45s/it]You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
`torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.a

2024-12-31 20:03:18,570 INFO image_writer.py:197 - writing: /media/smbshare/3Tpioneer_bids_predictions/sub-ms3024/ses-20161105/flair.t1_choroid_pineal_pituitary3_pred.nii.gz


Ensembling (rank 0)...:  99%|█████████▉| 140/141 [2:27:35<00:58, 58.57s/it]You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
`torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.a

2024-12-31 20:04:12,654 INFO image_writer.py:197 - writing: /media/smbshare/3Tpioneer_bids_predictions/sub-ms3028/ses-20210301/flair.t1_choroid_pineal_pituitary3_pred.nii.gz


Ensembling (rank 0)...: 100%|██████████| 141/141 [2:28:29<00:00, 63.19s/it]
